<a href="https://colab.research.google.com/github/chrisnelsonlab/BMEG4983/blob/master/W8_BMEG4983.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

####BMEG 4983 - Genome Engineering and Synthetic Biology - 
####For more information, check out the Nelson lab for Therapeutic Genome Engineering (https://nelsonlab.uark.edu/)

For image credits, see the linked URL

#8. Data Workshop 8 - NGS Analysis

The goal for today is to be able to work through a small pseduodata set for indel analysis

#8.1 What you need to solve this:
1. Your data set which includes an amplicon (.fa file) and your pseudodata from NGS (.fastq file)

2. We are going to filter our sequences. See below how we can do that.

#8.2 Importing
First, let's import everything we need. We will start with the first cell to get biopython in and working.
We will also import a few other things that we need

In [ ]:
#Uncomment this line the first time
!pip install biopython
#Import some things we might need
import Bio
from Bio import AlignIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
from Bio.Seq import Seq 
import time
import requests
#This line makes sure it works
print("Biopython version:", Bio.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.2 MB/s eta 0:00:00
Biopython version: 1.80


/usr/local/lib/python3.8/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [ ]:
from Bio import SeqIO, SearchIO, Entrez
from urllib.request import urlretrieve
import os
import sys
input_file = "unknown-sequence.fa"
#Here is where you can access your data
print("Check here for a link to your data: https://github.com/chrisnelsonlab/BMEG4983/tree/master/pseudofastq")

#Replace these with your gene
fastq_url="https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53.fastq"
fasta_url="https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53_NGS.fa"

#The code below pulls in the sequence data and creates an amplicon sequence (what we are comparing to)
#and a list that contains all the fastq data (seq_data)
input_file = "P53_seq.fa"

#Fetch fasta data for amplicon
if not os.path.exists(input_file):
    urlretrieve(fasta_url, input_file)
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)
record = SeqIO.read(input_file, "fasta")
print(record.seq)

#Fetch pseduo NGS data
input_file_fastq = "P53_NGS.fq"
if not os.path.exists(input_file_fastq):
      urlretrieve(fastq_url, input_file_fastq)
for record in SeqIO.parse(input_file_fastq, "fastq"):
    print(record.id)

Check here for a link to your data: https://github.com/chrisnelsonlab/BMEG4983/tree/master/pseudofastq
TP53_NGS
TTCCATAGGTCTGAAAATGTTTCCTGACTCAGAGGGGGCTCGACGCTAGGATCTGACTGCGGCTCCTCCATGGCAGTGACCCGGAAGGCAGTCTGGCTGCTGCAAGAGGAAAAGTGGGGATCCAGCATGAGACACTTCCAACCCTGGGTC
M03884:126:000000000-B89KP:1:1101:15380:1865
M03884:126:000000000-B89KP:1:1101:16566:1924
M03884:126:000000000-B89KP:1:1101:15425:1961
M03884:126:000000000-B89KP:1:1101:14368:1974
M03884:126:000000000-B89KP:1:1101:16902:2000
M03884:126:000000000-B89KP:1:1101:12133:2009
M03884:126:000000000-B89KP:1:1101:18423:2009
M03884:126:000000000-B89KP:1:1101:13290:2022
M03884:126:000000000-B89KP:1:1101:13517:2026
M03884:126:000000000-B89KP:1:1101:15911:2057
M03884:126:000000000-B89KP:1:1101:15929:2060
M03884:126:000000000-B89KP:1:1101:19135:2090
M03884:126:000000000-B89KP:1:1101:16952:2101
M03884:126:000000000-B89KP:1:1101:13855:2105
M03884:126:000000000-B89KP:1:1101:13864:2124
M03884:126:000000000-B89KP:1:1101:13267:2127
M03884:126:000000

#8.3 Filtering
Next we will filter the sequences based on an average score threshold. We will make a dictionary (scoredict) which can convert the symbols to a numerical score. Then we can write a function that averages that score (averagePHRED)

In [ ]:
scoredict = {
    '!':'0',	
    '\”':'1',	
    '#':'2',	
    '$':'3',	
    '%':'4',	
    '&':'5',	
    '\’':'6',	
    '(':'7',	
    ')':'8',	
    '*':'9',	
    '+':'10',	
    ',':'11',	
    '-':'12',	
    '.':'13',	
    '/':'14',	
    '0':'15',	
    '1':'16',	
    '2':'17',	
    '3':'18',	
    '4':'19',	
    '5':'20',	
    '6':'21',	
    '7':'22',	
    '8':'23',	
    '9':'24',	
    ':':'25',	
    ';':'26',	
    '<':'27',	
    '=':'28',	
    '>':'29',	
    '?':'30',	
    '@':'31',	
    'A':'32',	
    'B':'33',	
    'C':'34',	
    'D':'35',	
    'E':'36',	
    'F':'37',	
    'G':'38',	
    'H':'39',	
    'I':'40'
}
def averagePHRED(score):
    #Filter out by average score
    totalscore =0
    for bp in score:
        totalscore = totalscore+int(scoredict[bp])
    averagescore=totalscore/len(score)
    return averagescore

# 8.4 Save filtered sequences
Next, we will save the filtered sequences as a new fastq file. This will be exported into your google drive in the filename you choose below (outputfilename)

In [ ]:
#This section will filter out the sequences below a threshold
#It will then output a new file with the outputfilename.
min_read_quality = 35
seqcount = 0
filteredseqs = []
outputfilename = 'HBB_filtered.fastq'
with open(outputfilename, 'w') as writefile:
  for i in range(0,len(seq_data)):
    if(averagePHRED(seq_data[i][3])>min_read_quality):
      writefile.write(seq_data[i][0]+'\r\n')
      writefile.write(seq_data[i][1]+'\r\n')
      writefile.write(seq_data[i][2]+'\r\n')
      writefile.write(seq_data[i][3]+'\r\n')
      filteredseqs.append(seq_data[i][1])
      seqcount=seqcount+1


print('There are '+str(seqcount)+' sequences remaining after filtering')
print('Saving as '+outputfilename)
print('Check for the file output in the folder on the left side of google colab')


There are 100 sequences remaining after filtering
Saving as HBB_filtered.fastq
Check for the file output in the folder on the left side of google colab


# 8.5 Indel types
Finally, we will quantify the edits by using a biopython alignment algorithm. We will check for indel types and quantify. 

In [ ]:
edit_types = [] #A list of unique edit types
alledits = [] #A list of all edits
insertions = 0  #Counter for number of insertions
deletions = 0 #Counter for number of deletions
unedited = 0 #Counter for number that are unedited

#Loop through all filteredsequences
for filteredseq in filteredseqs:
  
  #Get alignment using biopython
  alignments = pairwise2.align.globalms(amplicon, filteredseq,2, -1, -5, -1,penalize_end_gaps=False)
  alignment =alignments[0] #Pick the best alignment

  #Uncomment to see the total alignment
  print(format_alignment(*alignment))

  #Check if there are insertions (dashes) in the amplicon meaning there is an insertion
  if(alignment[0].count("-",0,len(amplicon))>0):
    #print('insertion')
    insertions = insertions +1
    x=alignment[0].find("-",0,len(amplicon))
    edit = "amplicon: "+alignment[0][x-15:x+15]+'\n'+"sequence: "+alignment[1][x-15:x+15]
    alledits.append(edit)
    if edit not in edit_types:
      edit_types.append(edit) 
  
  #Check if there are deletions (dashes) in the sequence meaning there is an deletion
  elif(alignment[1].count("-",0,len(amplicon))>0):
    #print('deletion')
    deletions = deletions+1
    x=alignment[1].find("-",0,len(amplicon))
    edit = "amplicon: "+alignment[0][x-15:x+15]+'\n'+"sequence: "+alignment[1][x-15:x+15]
    alledits.append(edit)
    if edit not in edit_types:
      edit_types.append(edit) 
  
  #Otherwise, there may be a SNP but we will call them unedited for now
  else:
    #print('not edited')
    unedited=unedited+1

for footprint in edit_types:
  count = alledits.count(footprint)
  print('There are '+str(count)+' edits with the following footprint:')
  print(footprint)
  print('\n')
indelrate = (insertions+deletions)/(insertions+deletions+unedited)*100
print('for a total indel rate of: '+str(indelrate)+'%')

CTCACCACCAACTTCATCCACGTTCACCTTGCCCCACAGGGCAGTAACGGCAGACTTCTCCTCAGGAGTCAGATGCACCATGGTGTCTGTTTGAGGTTGCTAGTGAACACAGTTGTGTCAGAAGCAAATGTAAGCAATAGATGGCTCTGC----------
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||          ||||||||||||||||||||||||||||||||||||||||||||||||||||||||          
CTCACCACCAACTTCATCCACGTTCACCTTGCCCCACAGGGCAGTAACGGCAGACTTCTCCTCAGGAGTCAGATGCACCATGGT----------GGTTGCTAGTGAACACAGTTGTGTCAGAAGCAAATGTAAGCAATAGATGGCTCTGCCCTGACTTTT
  Score=266

CTCACCACCAACTTCATCCACGTTCACCTTGCCCCACAGGGCAGTAACGGCAGACTTCTCCTCAGGAGTCAGATGCACCATGGTGTCTGTTTGAGGTTGCTAGTGAACACAGTTGTGTCAGAAGCAAATGTAAGCAATAGATGGCTCTGC
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
CTCACCACCAACTTCATCCACGTTCACCTTGCCCCACAGGGCAGTAACGGCAGACTTCTCCTCAGGAGTCAGATGCACCATGGTGTCTGTTTGAGGTTGCTAGTGAACACAGTTGTGTCAGAAGCAAATGTAAGCAATAGATGGCTCTGC
  Score=300

CTCACCACCAACTTCATCCACGTTCACCTTGCCCCACA

#8.6 Other considerations

* We only examined a single read when we would most commonly used paired-end reads. Could you write a script to merge two reads? What would you need to do?
* Our algorithm doesn't count SNPs. Could you write something to find thsese SNPs?
* This data was very clean. What if there were sequencing errors? How would we handle these? 
* We elimiated entire sequences based on a score cut off. Our sequencing data records quality per base. Could we use this information to save some of the data?

#8.7 HW6 Analyzing fastq pseudo data for your gene
The following questions are based on data for your gene that should be on github:
https://github.com/chrisnelsonlab/BMEG4983/tree/master/pseudofastq

If your gene is not there, please email me and I will add it.



## 8.7.1 HW6 Q1 - Filtering

Generate a fasta file removing reads below an average quality of 35. 
You don't have to provide the fasta file for credit. Put the number of reads remaining and the number of reads filtered out below:

Notes for partial credit:



In [ ]:
Reads_remaining = 1
Reads_filtered_out = 1

## 8.7.2 HW6 Q2 Indel percentage
Write a script (or adapt the above script) to align sequences. Find if each sequence is a perfect match or has an indel. What is the overall indel percentage?

Notes for partial credit:



In [ ]:
#Number from 0 to 100
overall_indel_percentage = 0

## 8.7.3 HW6 Q3
Break down the most common indel types in this data set (HINT: there should be three types of indels in your data). Report the indel sequences and thier percentages. Make a smaller window for the data for deisplay (e.g. show 20 or so bp around the indel)

Notes for partial credit:



In [ ]:
#5 bp deletion
Indel_v1 = "ATGGCAGT-----GCAGTCGAT"
Indel_v1_percent = 24

#3 bp insertion CCC
Indel_v2 = "ATGGCAGTGACCCCAGGGAGAGGCAGTC"
Indel_v2_percent = 7

#3 bp deletion
Indel_v3 = "ATGGCAGTG---GGGAGAGGCAGTC"
Indel_v3_percent = 12

Overall_indel_percentage =43




## 8.7.4 HW6 Q4 - Which gRNA did we use?
What do you expect was the SpCas9 protospacer used in this experient? Put the 20 bp protospacer and PAM below. Assume that SpCas9 was used with an NGG PAM

Example solution: Based on the location of the indels the following two gRNAs are nearby and may have produced the indel footprint we detected

Protospacer1 = 'NNNNNNNNNNNNNNNNNNNN'
PAM1 = 'NGG'

Protospacer2 = 'NNNNNNNNNNNNNNNNNNNN'
PAM2 = 'NGG'

Because the cut site of protospacer 1 was exactly at the location of the 3 bp insertion, I expect protospacer 1 was most likely used

